In [31]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
from sklearn.neural_network import MLPClassifier
from numpy import random
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from numpy import matlib
import qgrid
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
import sys; sys.path.append('./')
from utils import Utils
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time
import qgrid

In [2]:
ut = Utils()

In [3]:
X, Y =ut.get_matrixes()

In [4]:
def select_features(modelo, n_features, fwd, fltg):
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=2,
           scoring='accuracy',
           cv=0,
           n_jobs=-1,
           pre_dispatch=10)
    
    return sfs

In [26]:
def trainSFS(n_features,clf):
    
    forward = True
    floating = False
   
        
        
    tiempo_i = time.time()
    
    #Implemetamos la metodología de validación 

    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)
    for train, test in skf.split(X_train_d,y_train_d):
        

        X_train, X_test = X[train], X[train]
        y_train, y_test = Y[train], Y[train]  
        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        sf = select_features(clf, n_features, forward, floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        print("iteracion: " + str(j))
        print("feature: " + str(n_features))
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1
    return  str(np.mean(Errores)), str(np.std(Errores)), str(sf.k_score_*100), str(time.time()-tiempo_i),sf.k_feature_idx_

In [27]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [28]:
def train(clf):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación

    j = 0
    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)

    for train, test in skf.split(X,Y):

        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X=X, Y=Y, train=train, test=test)

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(Xtrain,Ytrain)

        # Validación del modelo
        ypred = model.predict(Xtest)

        #######

        Errores[j] = classification_error(ypred, Ytest)
        j+=1
    
    m_errores = np.mean(Errores)
    ic_errores = np.std(Errores)
    eficiencia = (1 - np.mean(Errores))*100
    tEjec = time.time()-tiempo_i
                        
    print('finalizado clf sin selección',str(m_errores), str(ic_errores), str(eficiencia), str(tEjec))
    return  m_errores, str(np.std(Errores)), str((1 - np.mean(Errores))*100), str(time.time()-tiempo_i)

In [29]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['MLP sin Selección','MLP + SFS','MLP + SFS','MLP + SFS','MLP + SFS','MLP + SFS']),
    '# de características seleccionadas' : pd.Series([23, 3, 5, 10,15,20]),
   })
ic_ = []
tEjec_ = []
errorVal_ = []
eficiencia_ = []
caracteristicas=[]
model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = (20, 20), activation = 'relu', max_iter=500), n_jobs=-1)
for e, i in zip(df_types['Tecnica'], df_types['# de características seleccionadas']):
    
    if(e == "MLP sin Selección"):
        errorVal, ic, eficiencia, tEjec = train(model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
    else:
        print('entrenando: ',e,i)
        errorVal, ic, eficiencia, tEjec,car = trainSFS(i, model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
        caracteristicas.append(car)



finalizado clf sin selección 0.0670685619228169 0.024698612303261664 93.29314380771831 17.322251558303833
entrenando:  MLP + SFS 3
iteracion: 0
feature: 3


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   16.5s finished

[2021-01-16 23:22:01] Features: 1/3 -- score: 0.7754297994269341[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   39.5s finished

[2021-01-16 23:22:41] Features: 2/3 -- score: 0.8653295128939829[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   48.4s finished

[2021-01-16 23:23:29] Features: 3/3 -- score: 0.8889684813753582[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 1
feature: 3


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   16.0s finished

[2021-01-16 23:23:45] Features: 1/3 -- score: 0.7732808022922636[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   47.0s finished

[2021-01-16 23:24:32] Features: 2/3 -- score: 0.8470630372492837[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   56.0s finished

[2021-01-16 23:25:28] Features: 3/3 -- score: 0.8782234957020058[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 2
feature: 3


...

[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   16.5s finished

[2021-01-17 01:29:44] Features: 1/20 -- score: 0.7707736389684814[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   46.3s finished

[2021-01-17 01:30:30] Features: 2/20 -- score: 0.8384670487106017[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   48.8s finished

[2021-01-17 01:31:19] Features: 3/20 -- score: 0.8732091690544412[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  1.0min finished

[2021-01-17 01:32:19] Features: 4/20 -- score: 0.9036532951289399[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   54.3s finished

[2021-01-17 01:33:14] Features: 5/20 -- score: 0.9122492836676218[Parallel(n

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   17.0s finished

[2021-01-17 01:41:49] Features: 1/20 -- score: 0.7603868194842407[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   41.3s finished

[2021-01-17 01:42:31] Features: 2/20 -- score: 0.8445558739255015[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   50.1s finished

[2021-01-17 01:43:21] Features: 3/20 -- score: 0.8685530085959885[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   55.5s finished

[2021-01-17 01:44:16] Features: 4/20 -- score: 0.914756446991404[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   49.3s finished

[2021-01-17 01:45:06] Features: 5/20 -- score: 0.9215616045845272[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   15.3s finished

[2021-01-17 01:53:10] Features: 1/20 -- score: 0.786609380594343[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   37.8s finished

[2021-01-17 01:53:48] Features: 2/20 -- score: 0.8431793770139635[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   40.3s finished

[2021-01-17 01:54:29] Features: 3/20 -- score: 0.8725384890798424[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   53.4s finished

[2021-01-17 01:55:22] Features: 4/20 -- score: 0.8904403866809882[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   47.4s finished

[2021-01-17 01:56:09] Features: 5/20 -- score: 0.9119226638023631[Parallel(n_

In [32]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['MLP sin Selección','MLP + SFS','MLP + SFS','MLP + SFS','MLP + SFS','MLP + SFS']),
    '# de características seleccionadas' : pd.Series([23, 3, 5, 10,15,20]),
   })
df_types["Eficiencia(%)"] = eficiencia_
df_types["Error de validación"] = errorVal_
df_types["IC(std)"] = ic_
df_types["Tiempo de ejecución"] = tEjec_

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
df_types.to_csv('car_select_MLP.csv')
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

/home/juan/simulation/simulation/lib/python3.8/site-packages/qgrid/grid.py:1329: FutureWarning: inplace is deprecated and will be removed in a future version.
  df.index.set_levels(col_series, level=key_index, inplace=True)


Eficiencia(%)  \
Tecnica           # de características seleccionadas                      
MLP sin Selección 23                                  93.29314380771831   
MLP + SFS         3                                   87.21804511278195   
                  5                                   90.40458288578589   
                  10                                   93.3046902971715   
                  15                                   94.5578231292517   
                  20                                  95.88256355173648   

                                                       Error de validación  \
Tecnica           # de características seleccionadas                         
MLP sin Selección 23                                              0.067069   
MLP + SFS         3                                    0.11952652045586745   
                  5                                    0.08272779779986192   
                  10                                  0.044317070818675836   
                  15                                  0.037960743036469596   
                  20                                  0.031604479372807776   

                                                                   IC(std)  \
Tecnica           # de características seleccionadas                         
MLP sin Selección 23                                  0.024698612303261664   
MLP + SFS         3                                   0.006127726952674068   
                  5                                   0.009080248088162607   
                  10                                  0.014373896606855736   
                  15                                  0.010632462724323659   
                  20                                  0.007981861181809094   

                                                       Tiempo de ejecución  
Tecnica           # de características seleccionadas                        
MLP sin Selección 23                                  17.323665380477905 s  
MLP + SFS         3                                   455.81344652175903 s  
                  5                                   1236.5545377731323 s  
                  10                                  2796.0677461624146 s  
                  15                                  2453.3874883651733 s  
                  20                                   2812.185290336609 s